# 0 - Load modules

In [1]:
import sys
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import glob
from netCDF4 import Dataset
import cartopy.crs as ccrs
import json
import os
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
reg='GS'
config='GS36'


In [3]:
def plot_prof(maxdep,dm1,dm2,dm3,ref,temp_en4,salt_en4,mean_temp_model1,mean_salt_model1,mean_temp_model2,mean_salt_model2,mean_temp_model3,mean_salt_model3):


    fig, axs = plt.subplots(1,2, figsize=(10, 10))
    axs = axs.ravel()
    title = ref+' depen4 = '+str(d)+' dep1 = '+str(dm1)+' dep2 = '+str(dm2)+' dep3 = '+str(dm3)
    plt.suptitle(title,size = 25,y=1.05)

    axs[0].plot(mean_temp_model1,vert_standart,'r.-', label=reg+'36-75')
    axs[0].plot(mean_temp_model2,vert_standart,'b.-', label=reg+'36-150surf')
    axs[0].plot(mean_temp_model3,vert_standart,'g.-', label=reg+'36-150arctan')
    axs[0].plot(temp_en4,vert_standart,'k.-', label='EN4 obs')
    axs[0].set_ylabel('Depth [m]', size=14)
    axs[0].set_ylabel('Temperature [degC]', size=14)
    axs[0].grid(True, which='both')
    axs[0].set_ylim(maxdep, 0)
    axs[0].xaxis.tick_top()
    axs[0].xaxis.set_label_position('top') 

    axs[1].plot(mean_salt_model1,vert_standart,'r.-', label=reg+'36-75')
    axs[1].plot(mean_salt_model2,vert_standart,'b.-', label=reg+'36-150surf')
    axs[1].plot(mean_salt_model3,vert_standart,'g.-', label=reg+'36-150arctan')
    axs[1].plot(salt_en4,vert_standart,'k.-', label='EN4 obs')
    axs[1].set_ylabel('Depth [m]', size=14)
    axs[1].set_ylabel('Salinity [PSU]', size=14)
    axs[1].grid(True, which='both')
    axs[1].set_ylim(maxdep, 0)
    axs[1].xaxis.tick_top()
    axs[1].xaxis.set_label_position('top') 

    plt.legend()
    fig.tight_layout()
    plt.savefig('Plots/Plot_profile_'+ref+'GS-allsimus.png')


In [4]:
    namezone='NATL60-CJM165_'+reg+'_y2012-2013'
    jsonfile='/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_'+reg+'_y2012-2013/NATL60-CJM165_'+reg+'_y2012-2013_final.json'
    dirn1='/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_'+reg+'_y2012-2013/'+config+'-MPC001/'    
    dirn2='/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_'+reg+'_y2012-2013/'+config+'-MPC002/'    
    dirn3='/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_'+reg+'_y2012-2013/'+config+'-MPC003/'    
    
    #averaging the data in boxes

    vert_standart=[0,2,4,6,8,10,12,14,16,18,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,110,120,130,140,150,160,170,180,190,200,220,240,260,280,300,320,340,360,380,400,420,440,460,480,500,520,540,560,580,600,620,640,660,680,700,720,740,760,780,800,820,840,860,880,900,920,940,960,980,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000]

    def get_netcdf_profiles(netcdf_name):

        dsprof=xr.open_dataset(netcdf_name)
        depth=dsprof['depth_en4']
        temp_en4=dsprof['temp_profileEN4']
        salt_en4=dsprof['salt_profileEN4']
        mean_temp_model=dsprof['mean_temp_model']
        mean_salt_model=dsprof['mean_salt_model']
        percent10_temp_model=dsprof['percent10_temp_model']
        percent10_salt_model=dsprof['percent10_salt_model']
        percent90_temp_model=dsprof['percent90_temp_model']
        percent90_salt_model=dsprof['percent90_salt_model']

        return depth,temp_en4,salt_en4,mean_temp_model,mean_salt_model

    sourcefile=open(jsonfile,'rU')
    infos=json.load(sourcefile)
    nb_profilesEN4=len(infos)

    def get_ecart(temp_en4,salt_en4,mean_temp_model,mean_salt_model):
            ien4t=np.where(np.isnan(temp_en4)==True)
            ien4s=np.where(np.isnan(salt_en4)==True)
            imodt=np.where(np.isnan(mean_temp_model)==True)
            imods=np.where(np.isnan(mean_salt_model)==True)
            if len(imodt[:][0])>0:
                dep_modt=depth[np.min(imodt[:][0])].values
            else:
                dep_modt=2000
            if len(imods[:][0])>0:
                dep_mods=depth[np.min(imods[:][0])].values
            else:
                dep_mods=2000
            if len(ien4t[:][0]) >0: 
                dept=depth[np.min(ien4t[:][0])].values
            else:
                dept=2000
            
            if len(ien4s[:][0]) >0:
                deps=depth[np.min(ien4s[:][0])].values
            else:
                deps=2000
                
            return dept,deps,np.abs(dept-dep_modt),np.abs(deps-dep_mods)

    for prof in np.arange(nb_profilesEN4):
        list_profiles = infos.keys()
        reference = str(list(list_profiles)[prof])
        reference_profile=reference[-16:-1]
        outname1=dirn1+"/profiles_EN4-"+reference_profile+"_"+config+"-MPC001_TS_vert-stand.nc"
        outname2=dirn2+"/profiles_EN4-"+reference_profile+"_"+config+"-MPC002_TS_vert-stand.nc"
        outname3=dirn3+"/profiles_EN4-"+reference_profile+"_"+config+"-MPC003_TS_vert-stand.nc"

        if os.path.exists(outname1):
            depth,temp_en4,salt_en4,mean_temp_model1,mean_salt_model1 = get_netcdf_profiles(outname1)

        if os.path.exists(outname2):
            depth,temp_en4,salt_en4,mean_temp_model2,mean_salt_model2 = get_netcdf_profiles(outname2)

        if os.path.exists(outname3):
            depth,temp_en4,salt_en4,mean_temp_model3,mean_salt_model3 = get_netcdf_profiles(outname3)

        dept,deps,ecart_t1,ecart_s1=get_ecart(temp_en4,salt_en4,mean_temp_model1,mean_salt_model1)
        dept,deps,ecart_t2,ecart_s2=get_ecart(temp_en4,salt_en4,mean_temp_model2,mean_salt_model2)
        dept,deps,ecart_t3,ecart_s3=get_ecart(temp_en4,salt_en4,mean_temp_model3,mean_salt_model3)

        if np.abs(ecart_t1-ecart_t2) > 0 or np.abs(ecart_t1-ecart_t3) > 0 or np.abs(ecart_t3-ecart_t2) > 0:
                print('prof '+reference_profile+' dept ='+str(dept)+' ecart_t1 ='+str(ecart_t1)+' ecart_t2 = '+str(ecart_t2)+' ecart_t3 ='+str(ecart_t3))
        if np.abs(ecart_s1-ecart_s2) > 0 or np.abs(ecart_s1-ecart_s3) > 0 or np.abs(ecart_s3-ecart_s2) > 0:
                print('prof '+reference_profile+' deps ='+str(deps)+' ecart_s1 ='+str(ecart_s1)+' ecart_s2 = '+str(ecart_s2)+' ecart_s3 ='+str(ecart_s3))


prof A20121001-03173 dept =2000 ecart_t1 =100.0 ecart_t2 = 0 ecart_t3 =0
prof A20121001-03173 deps =2000 ecart_s1 =100.0 ecart_s2 = 0 ecart_s3 =0
prof A20121001-03175 dept =960.0 ecart_t1 =20.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20121001-03175 deps =960.0 ecart_s1 =20.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20121001-03182 dept =1900.0 ecart_t1 =0.0 ecart_t2 = 100.0 ecart_t3 =0.0
prof A20121001-03182 deps =1900.0 ecart_s1 =0.0 ecart_s2 = 100.0 ecart_s3 =0.0
prof A20121001-03196 dept =1800.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =50.0
prof A20121001-03196 deps =1800.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =50.0
prof A20121001-03200 dept =1800.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =50.0
prof A20121001-03200 deps =1800.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =50.0
prof A20121001-03206 dept =1050.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20121001-03206 deps =1050.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20121001-03208 dept =1800.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_

prof A20121002-40083 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121002-40083 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121002-40084 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121002-40084 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121002-40085 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121002-40085 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121002-40086 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121002-40086 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121002-40087 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121002-40087 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121002-40088 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121002-40088 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121002-6

prof A20121005-03651 dept =1700.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20121005-03651 deps =1700.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20121005-03652 dept =1700.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20121005-03652 deps =1700.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20121005-03655 dept =1650.0 ecart_t1 =0.0 ecart_t2 = 50.0 ecart_t3 =0.0
prof A20121005-03655 deps =1650.0 ecart_s1 =0.0 ecart_s2 = 50.0 ecart_s3 =0.0
prof A20121005-03669 dept =1900.0 ecart_t1 =0.0 ecart_t2 = 100.0 ecart_t3 =0.0
prof A20121005-03669 deps =1900.0 ecart_s1 =0.0 ecart_s2 = 100.0 ecart_s3 =0.0
prof A20121005-03678 dept =1800.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =50.0
prof A20121005-03678 deps =1800.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =50.0
prof A20121005-03679 dept =1800.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =50.0
prof A20121005-03679 deps =1800.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =50.0
prof A20121005-03685 dept =1050.0 ecart_t1 =50.0 ecart_t2 

prof A20121006-41649 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121006-41649 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121006-41650 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121006-41650 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121006-41651 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121006-41651 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121006-41652 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121006-41652 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121006-41653 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121006-41653 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121006-41654 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20121006-41654 deps =200.0 ecart_s1 =150.0 ecart_s2 = 145.0 ecart_s3 =145.0
prof A20121006-4

prof A20121008-03993 dept =1700.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20121008-03993 deps =1700.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20121008-04000 dept =1900.0 ecart_t1 =0.0 ecart_t2 = 100.0 ecart_t3 =0.0
prof A20121008-04000 deps =1900.0 ecart_s1 =0.0 ecart_s2 = 100.0 ecart_s3 =0.0
prof A20121008-04004 dept =1850.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20121008-04004 deps =1850.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20121008-04013 dept =960.0 ecart_t1 =20.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20121008-04013 deps =960.0 ecart_s1 =20.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20121008-04039 dept =1050.0 ecart_t1 =50.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20121008-04039 deps =1050.0 ecart_s1 =50.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20121008-04043 dept =800.0 ecart_t1 =20.0 ecart_t2 = 20.0 ecart_t3 =0.0
prof A20121008-04043 deps =800.0 ecart_s1 =20.0 ecart_s2 = 20.0 ecart_s3 =0.0
prof A20121008-04050 deps =1900.0 ecart_s1 =0.0 ecart_s2 = 100.0

...

prof A20130925-81760 dept =150.0 ecart_t1 =95.0 ecart_t2 = 90.0 ecart_t3 =95.0
prof A20130925-81760 deps =150.0 ecart_s1 =95.0 ecart_s2 = 90.0 ecart_s3 =95.0
prof A20130925-81763 dept =80.0 ecart_t1 =5.0 ecart_t2 = 5.0 ecart_t3 =0.0
prof A20130925-81763 deps =80.0 ecart_s1 =5.0 ecart_s2 = 5.0 ecart_s3 =0.0
prof A20130925-81764 dept =80.0 ecart_t1 =5.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20130925-81764 deps =80.0 ecart_s1 =5.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20130925-81765 dept =80.0 ecart_t1 =5.0 ecart_t2 = 0.0 ecart_t3 =0.0
prof A20130925-81765 deps =80.0 ecart_s1 =5.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20130925-81794 dept =160.0 ecart_t1 =105.0 ecart_t2 = 100.0 ecart_t3 =105.0
prof A20130925-81794 deps =160.0 ecart_s1 =105.0 ecart_s2 = 100.0 ecart_s3 =105.0
prof A20130925-81795 dept =70.0 ecart_t1 =15.0 ecart_t2 = 10.0 ecart_t3 =15.0
prof A20130925-81795 deps =70.0 ecart_s1 =15.0 ecart_s2 = 10.0 ecart_s3 =15.0
prof A20130925-81798 dept =70.0 ecart_t1 =35.0 ecart_t2 = 40.0 ecart_t3 

prof A20130925-82612 dept =150.0 ecart_t1 =80.0 ecart_t2 = 70.0 ecart_t3 =75.0
prof A20130925-82612 deps =150.0 ecart_s1 =80.0 ecart_s2 = 70.0 ecart_s3 =75.0
prof A20130925-82613 dept =160.0 ecart_t1 =90.0 ecart_t2 = 80.0 ecart_t3 =85.0
prof A20130925-82613 deps =160.0 ecart_s1 =90.0 ecart_s2 = 80.0 ecart_s3 =85.0
prof A20130925-82625 dept =75.0 ecart_t1 =35.0 ecart_t2 = 40.0 ecart_t3 =35.0
prof A20130925-82625 deps =75.0 ecart_s1 =35.0 ecart_s2 = 40.0 ecart_s3 =35.0
prof A20130925-82626 dept =75.0 ecart_t1 =35.0 ecart_t2 = 40.0 ecart_t3 =35.0
prof A20130925-82626 deps =75.0 ecart_s1 =35.0 ecart_s2 = 40.0 ecart_s3 =35.0
prof A20130925-82665 dept =160.0 ecart_t1 =90.0 ecart_t2 = 80.0 ecart_t3 =85.0
prof A20130925-82665 deps =160.0 ecart_s1 =90.0 ecart_s2 = 80.0 ecart_s3 =85.0
prof A20130925-82666 dept =160.0 ecart_t1 =90.0 ecart_t2 = 80.0 ecart_t3 =85.0
prof A20130925-82666 deps =160.0 ecart_s1 =90.0 ecart_s2 = 80.0 ecart_s3 =85.0
prof A20130925-82680 dept =75.0 ecart_t1 =35.0 ecart_t2 

prof A20130926-06568 dept =980.0 ecart_t1 =40.0 ecart_t2 = 20.0 ecart_t3 =20.0
prof A20130926-06568 deps =980.0 ecart_s1 =40.0 ecart_s2 = 20.0 ecart_s3 =20.0
prof A20130926-06580 dept =580.0 ecart_t1 =515.0 ecart_t2 = 520.0 ecart_t3 =520.0
prof A20130926-06580 deps =580.0 ecart_s1 =515.0 ecart_s2 = 520.0 ecart_s3 =520.0
prof A20130926-47013 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130926-47013 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130926-47014 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130926-47014 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130926-47015 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130926-47015 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130926-47018 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130926-47018 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130926-47019 dept =200.0 ecart_t1 

prof A20130927-40315 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130927-40315 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130927-40316 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130927-40316 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130927-40317 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130927-40317 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130927-47024 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130927-47024 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130927-47025 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130927-47025 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130927-47026 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130927-47026 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130927-47027 dept =200.0 ecart_t1 =150

prof A20130929-40341 dept =200.0 ecart_t1 =150.0 ecart_t2 = 145.0 ecart_t3 =145.0
prof A20130929-40341 deps =2.0 ecart_s1 =48.0 ecart_s2 = 53.0 ecart_s3 =53.0
prof A20130929-64111 dept =190.0 ecart_t1 =150.0 ecart_t2 = 155.0 ecart_t3 =155.0
prof A20130929-64111 deps =190.0 ecart_s1 =150.0 ecart_s2 = 155.0 ecart_s3 =155.0
prof A20130929-64113 deps =70.0 ecart_s1 =5.0 ecart_s2 = 0.0 ecart_s3 =0.0
prof A20130929-64115 dept =150.0 ecart_t1 =105.0 ecart_t2 = 105.0 ecart_t3 =110.0
prof A20130929-64115 deps =150.0 ecart_s1 =105.0 ecart_s2 = 105.0 ecart_s3 =110.0
prof A20130929-64116 dept =110.0 ecart_t1 =40.0 ecart_t2 = 35.0 ecart_t3 =35.0
prof A20130929-64116 deps =110.0 ecart_s1 =40.0 ecart_s2 = 35.0 ecart_s3 =35.0
prof A20130929-64117 dept =75.0 ecart_t1 =10.0 ecart_t2 = 15.0 ecart_t3 =15.0
prof A20130929-64117 deps =75.0 ecart_s1 =10.0 ecart_s2 = 15.0 ecart_s3 =15.0
prof A20130929-64118 dept =110.0 ecart_t1 =40.0 ecart_t2 = 35.0 ecart_t3 =35.0
prof A20130929-64118 deps =110.0 ecart_s1 =40